In [1]:
!pip install mesa numpy scapy pandas matplotlib
!sudo apt-get install mininet tcpreplay

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.5/226.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  cgroup-tools iperf libcgroup1 libpcap0.8 libunbound8 libwrap0 net-tools
  netbase openvswitch-common openvswitch-switch python3-netifaces
  python3-openvswitch python3-sortedcontainers socat telnet uuid-runtime
Suggested packages:
  ethtool openvswitch-doc python-sortedcontainers-doc
The following NEW packages will be installed:
  cgroup-tools iperf libcgroup1 libpcap0.8 libunbound8 libwrap0 mininet
  net-tools netbase openvswitch-common openvswitch-switch python3-netifaces
  python3-openvswitch python3-sortedcontainers socat tcpreplay telnet
  uuid-runtime
0 upgraded, 18 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,536 kB of archives.
After this operation, 16.1 

In [11]:
model = NetworkModel(n_nodes=10, n_malicious=2)
for i in range(5):
    model.step()
    print(model.traffic)


[{'node': 0, 'type': 'malicious'}, {'node': 2, 'type': 'normal'}, {'node': 4, 'type': 'normal'}, {'node': 9, 'type': 'normal'}, {'node': 6, 'type': 'normal'}, {'node': 3, 'type': 'normal'}, {'node': 5, 'type': 'normal'}, {'node': 1, 'type': 'malicious'}, {'node': 7, 'type': 'normal'}, {'node': 8, 'type': 'normal'}]
[{'node': 7, 'type': 'normal'}, {'node': 2, 'type': 'normal'}, {'node': 5, 'type': 'normal'}, {'node': 9, 'type': 'normal'}, {'node': 3, 'type': 'normal'}, {'node': 4, 'type': 'normal'}, {'node': 0, 'type': 'malicious'}, {'node': 1, 'type': 'malicious'}, {'node': 6, 'type': 'normal'}, {'node': 8, 'type': 'normal'}]
[{'node': 9, 'type': 'normal'}, {'node': 1, 'type': 'malicious'}, {'node': 4, 'type': 'normal'}, {'node': 6, 'type': 'normal'}, {'node': 5, 'type': 'normal'}, {'node': 2, 'type': 'normal'}, {'node': 8, 'type': 'normal'}, {'node': 7, 'type': 'normal'}, {'node': 3, 'type': 'normal'}, {'node': 0, 'type': 'malicious'}]
[{'node': 8, 'type': 'normal'}, {'node': 2, 'type

<ipython-input-10-d310ab926bc6>:25: DeprecationWarning: The time module and all its Schedulers are deprecated and will be removed in Mesa 3.1. They can be replaced with AgentSet functionality. See the migration guide for details. https://mesa.readthedocs.io/latest/migration_guide.html#time-and-schedulers
  self.schedule = RandomActivation(self)


In [12]:
import numpy as np

transition_matrix = np.array([
    [0.7, 0.2, 0.1],  # Normal to (Normal, Suspicious, Malicious)
    [0.4, 0.5, 0.1],  # Suspicious to (Normal, Suspicious, Malicious)
    [0.1, 0.3, 0.6]   # Malicious to (Normal, Suspicious, Malicious)
])

def next_state(current_state, transition_matrix):
    return np.random.choice([0, 1, 2], p=transition_matrix[current_state])

def generate_states(initial_state, steps, transition_matrix):
    states = [initial_state]
    for _ in range(steps - 1):
        states.append(next_state(states[-1], transition_matrix))
    return states


In [14]:
from mesa import Model
from mesa.agent import Agent
from mesa.time import RandomActivation
from mesa.space import SingleGrid
import random

class NetworkNode(Agent):
    def __init__(self, unique_id, model, is_malicious=False):
        super().__init__(model)
        self.unique_id = unique_id
        self.is_malicious = is_malicious

    def step(self):
        self.state = next_state(self.state, self.model.transition_matrix)
        traffic_type = ["normal", "suspicious", "malicious"][self.state]
        self.model.traffic.append({"node": self.unique_id, "type": traffic_type})

class NetworkModel(Model):
    def __init__(self, n_nodes, n_malicious, transition_matrix):
        super().__init__()  # Initialize the Model base class
        self.n_nodes = n_nodes
        self.n_malicious = n_malicious
        self.random = random.Random()
        self.schedule = RandomActivation(self)
        self.transition_matrix = transition_matrix
        self.grid = SingleGrid(10, 10, torus=False)
        self.traffic = []

        # Add nodes
        for i in range(n_nodes):
            is_malicious = i < n_malicious
            node = NetworkNode(i, self, is_malicious)
            self.schedule.add(node)

    def step(self):
        self.traffic = []
        self.schedule.step()


In [16]:
from mininet.node import OVSController


NameError: name 'topo' is not defined

In [26]:
from mininet.net import Mininet
from mininet.topo import Topo
from mininet.link import TCLink
from mininet.node import RemoteController

class SimpleTopo(Topo):
    def build(self):
        switch = self.addSwitch('s1')
        h1 = self.addHost('h1')
        h2 = self.addHost('h2')
        self.addLink(h1, switch)
        self.addLink(h2, switch)

net = Mininet(topo=SimpleTopo(), link=TCLink, controller = RemoteController)
# net = Mininet(topo=SimpleTopo())
net.start()
net.pingAll()
net.stop()


Unable to contact the remote controller at 127.0.0.1:6653

Unable to contact the remote controller at 127.0.0.1:6633

Setting remote controller to 127.0.0.1:6653

Error setting lo up: SIOCSIFFLAGS: Operation not permitted
 ERROR:mininet:Error setting lo up: SIOCSIFFLAGS: Operation not permitted
 


Exception: Error creating interface pair (h1-eth0,s1-eth1): RTNETLINK answers: Operation not permitted
 

In [23]:
!sudo modprobe openvswitch


modprobe: FATAL: Module openvswitch not found in directory /lib/modules/6.1.85+


In [24]:
!sudo apt install linux-headers-$(uname -r)
!git clone https://github.com/openvswitch/ovs.git
!cd ovs
!./boot.sh
!./configure --with-linux=/lib/modules/$(uname -r)/build
!make
!sudo make install
!sudo make modules_install
!sudo modprobe openvswitch


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package linux-headers-6.1.85
E: Couldn't find any package by glob 'linux-headers-6.1.85'
Cloning into 'ovs'...
remote: Enumerating objects: 178282, done.
remote: Counting objects: 100% (2264/2264), done.
remote: Compressing objects: 100% (650/650), done.
remote: Total 178282 (delta 1806), reused 1737 (delta 1560), pack-reused 176018 (from 1)
Receiving objects: 100% (178282/178282), 126.45 MiB | 24.16 MiB/s, done.
Resolving deltas: 100% (141589/141589), done.
/bin/bash: line 1: ./boot.sh: No such file or directory
/bin/bash: line 1: ./configure: No such file or directory
make: *** No targets specified and no makefile found.  Stop.
make: *** No rule to make target 'install'.  Stop.
make: *** No rule to make target 'modules_install'.  Stop.
modprobe: FATAL: Module openvswitch not found in directory /lib/modules/6.1.85+


In [25]:
!sudo apt update
!sudo apt install linux-headers-$(uname -r)


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,188 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,527 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/